# Risk Metrics Analysis

This notebook demonstrates how to calculate common risk metrics such as Sharpe ratio, Value at Risk (VaR), and max drawdown using simulated price data and the BacktestEngine from the `src` module.

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from src.backtest import BacktestEngine

# Helper to generate price data
def make_price_df(days=252, start_price=100.0):
    dates = [datetime(2025, 1, 1) + timedelta(days=i) for i in range(days)]
    prices = start_price * (1 + np.cumsum(np.random.normal(0, 0.001, size=days)))
    return pd.DataFrame({'close': prices}, index=dates)

# Simulate price series and backtest
price_df = make_price_df()
def simple_momentum(df):
    return pd.Series(np.where(df['close'].pct_change().fillna(0) > 0, 1, 0), index=df.index)

engine = BacktestEngine(price_data=price_df, strategy_fn=simple_momentum)
equity = engine.run()
returns = equity['total'].pct_change().dropna()

# Display head of equity curve
equity.head()

## Sharpe Ratio
Sharpe Ratio is calculated as:

$$\text{Sharpe} = \frac{\sqrt{252} \times \bar{R}}{\sigma_R}$$


In [ ]:
# Calculate Sharpe ratio
sharpe_ratio = np.sqrt(252) * returns.mean() / returns.std()
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

## Value at Risk (VaR)
The 95% VaR is the 5th percentile of the return distribution.

In [ ]:
# Calculate VaR at 95%
var_95 = np.percentile(returns, 5)
print(f"95% VaR: {var_95:.4f}")

## Maximum Drawdown
Drawdown is the peak-to-trough decline. We compute it on the equity curve.

In [ ]:
# Compute drawdown
cum_max = equity['total'].cummax()
drawdown = (equity['total'] - cum_max) / cum_max
max_drawdown = drawdown.min()
print(f"Maximum Drawdown: {max_drawdown:.4f}")

### Interpretation
- A **Sharpe Ratio** above 1.0 is generally considered acceptable.
- A **VaR** of -0.02 means a 2% loss at the 95% confidence level.
- **Max Drawdown** indicates the worst observed loss from a peak.